<div style="text-align:center; margin-top: 1cm;">
    <h2>Київський політехнічний інститут імені Ігоря Сікорського</h2>
    <h2>Приладобудівний факультет</h2>
    <h3>Кафедра автоматизації та систем неруйнівного контролю</h3>
    <br><br><br>
</div>

<div style="text-align:center; margin-top: 5cm;">
    <h2>Практична робота №2 </h2>
    <h2>з предмету "Комп'ютерний зір"</h2>
</div>

<div style="text-align:right; margin-top: 5cm;">
<p>Студент: Погорєлов Богдан<br>
    Група: ПК-51мп<br>
</p>
</div>
<div style="text-align:center; margin-top: 5cm;">
2025 рік  <br><br><br><br>
</div>

Розробити програму, яка виконує такі дії з ЦЗ:
1. В зоні вказаної геометричної форми виконати порогову обробку. Порогове значення вибрати самостійно
так, щоб найкращім чином виділити об’єкти. Порівняти це значення з значенням, розрахованим при
застосуванні порогової обробки за методом Отсу. Результат порогової обробки вставити в вхідне ЦЗ в зоні
заданої геометричної форми. Площа цієї зони повинна перевищувати 50 % від площі цифрового
зображення. Геометрична форма цієї зони визначається по варіантам згідно Таблиці 1.
2. У ЦЗ виділити контури об’єктів за допомогою цифрових фільтрів Собеля та DoG (різниця гаусових функцій)
та порогової обробки. Порівняти результати роботи детектора контурів Сanny з вказаною вище
попередньою обробкою та без неї. 

Таблиця 1. Варіанти практичного завдання № 1, 2.

| Тип функції | лемніската Бернуллі | фігура Ліссажу | епіциклоїда | гіпоциклоїда | кардіоїда |
| :--- | :---: | :---: | :---: | :---: | :---: |
| **гармонічна** | 1 | 2 | 3 | 4 | 5 |
| **лінійна** | 6 | 7 | 8 | 9 | 10 |
| **гаусоїда** | 11 | 12 | 13 | 14 | 15 |
| **логарифмічна** | 16 | 17 | 18 | 19 | 20 |
| **кубічна** | 21 | 22 | 23 | 24 | 25 |